# ニューラルネットワークの学習

## データ駆動

機械学習においてはデータが命である

データからパターンを見つけ出し、データから答えを導く出すのが機械学習の本質である

ここで、入力データからデータの本質的なパターンを導き出す変換器を**特徴量**と呼ぶ

### 特徴量の抽出の歴史
最も単純な分類である線形分類においては、特徴量を抽出する必要はなく、入力データから有限回の学習により問題を解くことが可能だった

これは最初に行ったパーセプトロンによるAND, OR等の演算子の実装において証明されたように「パーセプトロンの収束原理」として知られている

一方、非線形分類問題は入力データからの自動的な学習が不可能であり、特徴量を抽出する必要が出てきた

当初は、特徴量の抽出からデータの学習（アルゴリズムの実装）まで全て人の手で行われており、このモデルは**エクスパートシステム**と呼ばれる

その後、特徴量はデータサイエンティストと呼ばれる人々によって抽出されるようになり、抽出された特徴量を**機械学習アルゴリズム**（SVM, kNNなど）を用いて自動的に学習することが可能になった

そして近年話題となった**ニューラルネットワーク（ディープラーニング）**においては、特徴量の抽出からデータの学習までを全てコンピュータによって行われるようになっている

![machine_learning_history](./img/machine_learning_history.png)

### パラメータの最適化と損失関数
機械学習およびニューラルネットワークにおいては、データの学習とはすなわち**パラメータの最適化**である

例えばニューラルネットワークは、重みとバイアスというパラメータの値を少しずつ変化させ、計算結果が教師データと近い値になるように学習を行っている

この時、計算結果と教師データとの誤差を表現する関数を**損失関数**と呼び、この損失関数を最小化することが学習の目的となる

関数の最小値を求める場合、数学的には微分という手法が用いられる

微分値とは「ある瞬間における変化の量」と定義される値のことであり、わかりやすく言えば、グラフの接線の傾きのことである

簡単のため二次関数を例にすると下図の通り、微分値が0になる（接線の傾きが0になる）点が関数の最小値点であり、パラメータの最適点であると考えることができる

![loss_function_optimization](./img/loss_function_optimization.png)